<h1 align="center">Feature Engineering</h1>

**Feature Engineering** is a concept that we run into across all chapters of this book. In each chapter, we run into different aspects of feature engineering. This notebook exists to collect all related knowledge and tips in one place.

## General tips

+ use your domain knowledge of the problem

+ search for features in existing literature and papers written by experts in the domain

<br/>
<br/>
## Make real-valued features dimensionless

Often, features measure different aspects of a system and are likely to have very different ranges and units. Ranges of some features may even be orders of magnitude higher or lower than other features. Such features will tend to dominate over other features and bias the predicted values.

What we need is a way to reduce all features to a common scale, or in other words, _normalize_.

One way to normalize features is to convert them to their _z-scores_. The z-score of a variable reduces its value to its distance from that variable's mean, expressed as number of standard deviation units:

$$ f' = {{f - \mu} \over \sigma } $$

Note that

+ $\mu$ is the feature's mean calculated using <span style="color:red">only the training data</span>

+ $\sigma$ is the feature's SD calculated using <span style="color:red">only the training data</span>

+ When normalizing the test set's features, the same <span style="color:red">training set's mean and SD</span> should be used.

<br/>

** Z-scores in scikit-learn **

Use [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from _preprocessing_ module:


```python
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
norm_train_x, norm_train_y = scaler.fit_transform(train_x, train_y)
norm_test_x = scaler.transform(test_x)

```


<br/>
<br/>

## Convert nominal variables into numeric variables

There are 2 ways to do this:

### <span style="color:green">1. Label encoding</span>
Convert each value of nominal variable into a distinct integer. Use [sklearn.preprocessing.LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) for this.

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit_transform(['classA', 'classB', 'classC', 'classA'])

array([0, 1, 2, 0])

An unfortunate side effect of this - which may cause problems for some ML algorithms as we'll see later - is that the converted values have an implicit ordinal relationship (0 < 1 < 2, ie classA < classB < classC) which may not be desirable if the variable is not ordinal by nature. 

<br/>
### <span style="color:green">2. One Hot encoding</span>

This converts each value into a vector of booleans where all elements of vector are zeroes except the element corresponding to the nominal value.

Effectively, it splits a single feature into multiple features. This is very advantageous for algorithms because now each of the new features can have its own weight and decision boundary, making discriminative algorithms - especially linear ones like Logistic and SVM - much more accurate. Why exactly this happens is explained in next section.

Use [sklearn.preprocessing.OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) for this.

For example, imagine a categorical column has been first encoded using `LabelEncoder` into the encoded labels [0, 1, 2, 0]. These can then be one-hot encoded to:

In [14]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False) # sparse is False here just to display a full matrix.
print(ohe.fit_transform([[0], [1], [2], [0]]))

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]]


One hot encoding does not suffer label encoding's problem of implicitly making a variable ordinal.


<br/>

### <span style="color:green">3. Choosing between Label and One Hot encoding</span>

Briefly
+ one hot encoding is better for features

+ label encoding should be restricted to response variables (in classification).

For simplifying the understanding of why label encoding is unsuitable for features, imagine a model with just one nominal feature $x$ encoded as 0,1,2. 

If the model is a linear model - such as linear regression, logistic regression, SVM, etc - we get linear decision boundaries like below to discriminate between classes. (As a side note, LR and SVM can give decision boundaries which are non-parallel to the axes, though it's not relevant to the reasoning here).

![decision boundary](images/logistic-regression-decision-boundary.png)

Since there is only 1 feature and 1 weight, there is only 1 decision boundary line. All observations on one side of this line are classified as one of the labels while all those on other side are classified as "not that label". But "not that label" is ambiguous because both the remaining labels qualify.

This explains why label encoding is not suitable for features that can take more than 2 labels and when using a linear model.

One hot encoding does not have this problem because there are as many features and weights as there are labels, and so the classifier can come up with a different decision boundary line for each of the labels.

However, decision trees and random forests, despite being linear models, can handle label encoded features especially when there is lots of data. Exactly why this is, I'm not able to explain intuitively (TODO). 

Similarly, algorithms which use similarity metrics like KNN, KMeans, etc will get confused by label encodings. For example, the "distance" between "red" and "blue" is 1 while that between "red" and "green" is 2, but blue is not more "similar" to red than green. This is meaningless since the variable is not ordinal by nature. Instead each label should be equidistant from every other label, and this is possible by one hot encoding.

Since one hot encoding can blow up the number of features, dimensionality reduction techniques like PCA should also be applied to the pipeline to use only the most variant subset of explanatory features.

** References: **

+ http://stackoverflow.com/questions/17469835/one-hot-encoding-for-machine-learning

+ https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science

+ https://www.quora.com/What-are-the-advantages-of-logistic-regression-over-decision-trees

+ http://datascience.stackexchange.com/questions/9443/when-to-use-one-hot-encoding-vs-labelencoder-vs-dictvectorizor